In [2]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

import cufflinks as cf

import pandas as pd
import numpy as np

import geojson


import sqlite3
from sqlalchemy import create_engine

In [3]:
geo_neigh = pd.read_json("neighbourhoods.geojson")
geo_neigh.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."


In [4]:
with open('neighbourhoods.geojson', 'r') as f:
    neigh = geojson.load(f)
    f.close()

prepare neighbourhoods to be placed on a map

In [5]:
lons=[]
lats=[]
neighbourhood_names=[]
for k in range(len(neigh['features'])):
    county_coords=np.array(neigh['features'][k]['geometry']['coordinates'][0][0])
    m, M =county_coords[:,0].min(), county_coords[:,0].max()
    lons.append(str(0.5*(m+M)))
    m, M =county_coords[:,1].min(), county_coords[:,1].max()
    lats.append(str(0.5*(m+M)))
    neighbourhood_names.append(neigh['features'][k]['properties']['neighbourhood'])

In [6]:
neighbourhood_names

['Batignolles-Monceau',
 'Palais-Bourbon',
 'Buttes-Chaumont',
 'Opéra',
 'Entrepôt',
 'Gobelins',
 'Vaugirard',
 'Reuilly',
 'Louvre',
 'Luxembourg',
 'Élysée',
 'Temple',
 'Ménilmontant',
 'Panthéon',
 'Passy',
 'Observatoire',
 'Popincourt',
 'Bourse',
 'Buttes-Montmartre',
 'Hôtel-de-Ville']

Check the 'listings.csv' file for prices

In [9]:
listings = pd.read_csv('listings.csv')

In [10]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2577,Loft for 4 by Canal Saint Martin,2827,Karine,NaN,Entrepôt,48.869933,2.362511,Entire home/apt,125,3,1,2019-01-02,0.86,1,85
1,5396,Explore the heart of old Paris,7903,Borzou,NaN,Hôtel-de-Ville,48.851001,2.358690,Entire home/apt,115,1,156,2019-02-03,1.33,1,129
2,7397,MARAIS - 2ROOMS APT - 2/4 PEOPLE,2626,Franck,NaN,Hôtel-de-Ville,48.857576,2.352751,Entire home/apt,115,10,236,2019-01-27,2.47,1,211
3,7964,Large & sunny flat with balcony !,22155,Anaïs,NaN,Opéra,48.874642,2.343411,Entire home/apt,99,3,6,2015-09-14,0.06,1,219
4,8522,GREAT FLAT w/ CITY VIEW,25435,Adrien,NaN,Ménilmontant,48.865279,2.393263,Entire home/apt,90,3,1,2010-06-16,0.01,1,365


In [11]:
map_text = (
    pd
    .DataFrame(neighbourhood_names)
    .set_index(0)
    .join(
        listings
        .groupby(['neighbourhood'])
        .agg(['count', 'median'])
        .price
        )
    .reset_index()
    .rename({0: 'neighbourhood'}, axis = 1)
    .apply(lambda x: x.neighbourhood + '<br>' + str(x['count']) + ' listings<br>median price: $' + str(x['median']), axis = 1)
    .values
)
map_text

array(['Batignolles-Monceau<br>3897 listings<br>median price: $75',
       'Palais-Bourbon<br>1642 listings<br>median price: $110',
       'Buttes-Chaumont<br>3271 listings<br>median price: $60',
       'Opéra<br>2657 listings<br>median price: $90',
       'Entrepôt<br>4056 listings<br>median price: $77',
       'Gobelins<br>2057 listings<br>median price: $68',
       'Vaugirard<br>4321 listings<br>median price: $80',
       'Reuilly<br>2343 listings<br>median price: $70',
       'Louvre<br>1116 listings<br>median price: $120',
       'Luxembourg<br>1720 listings<br>median price: $119',
       'Élysée<br>1491 listings<br>median price: $130',
       'Temple<br>2801 listings<br>median price: $108',
       'Ménilmontant<br>3451 listings<br>median price: $60',
       'Panthéon<br>1976 listings<br>median price: $95',
       'Passy<br>2903 listings<br>median price: $100',
       'Observatoire<br>2215 listings<br>median price: $70',
       'Popincourt<br>5731 listings<br>median price: $71',
 

show map

In [12]:
with open('../.mapbox.token', 'r') as f:
    mapbox_access_token = f.readline().strip()
    f.close()

data = [
    dict(
        type='scattermapbox',
        lat=lats,
        lon=lons,
        mode='markers',
        marker=dict(size=1, color='rgba(0,0,0,0)'),
        text=map_text,
        hoverinfo='text',
    )
]

layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = neigh,
                type = 'line',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=48.8530,
            lon=2.3499
        ),
        pitch=0,
        zoom=10.7,
        style='light'
    ),
    title = 'Neighbourhoods of Paris'
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='paris_map_1')